# Installing neccessary packages


# Install your packages here

In [3]:
install.packages('jsonlite')
install.packages('rvest')
install.packages('tidyverse')
install.packages('lubridate')
install.packages('httr')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("jsonlite"):
“installation of package ‘jsonlite’ had non-zero exit status”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("httr"):
“installation of package ‘httr’ had non-zero exit status”


# Load the libraries here

In [106]:
library(jsonlite)
library(httr)
library(rvest)
library(tidyverse)
library(lubridate)

# trimStringArray function

In [107]:
trimStringArray = function(array) {
  if (is.array(array)) {
    for (i in 1:length(array())) {
      array[i] = trimws(array[i])
    }
  } else {
    array = trimws(array)
  }
  array
}

# ternaryOperatorFunction | Vectorized function

In [109]:
ternary = function(expression_1, expression_2, condition) {
    if(condition) {
        expression_1
    } else {
        expression_2
    }
}

ternary_v = Vectorize(ternary)

# Create a vectorized version of strsplit()

In [110]:
strsplit_custom = function (string, splitter, index) {
    strsplit(string, splitter)[[1]][index]
}

strsplit_custom_v = Vectorize(strsplit_custom)

# Scrape Teilnehmer von WM

In [226]:
scrapeColumnsWM = function(game_name) {

  # base url

  url = 'https://www.transfermarkt.com/weltmeisterschaft-2022/teilnehmer/pokalwettbewerb/WM22/saison_id/2021'

  # read url

  url %>%
    read_html() -> html

  # variables

  columnValues = list() # a list containing all values
  columnNames = NULL
  box = NULL # searching for the table_header
  slugs = c()
  club_ids = c()

  # Obtain the correct responsive-table by name of table_header

  html %>%
    html_elements('.box') -> boxes

  # Print the table headers

  boxes

  # Iterate over each table header to find the match with table_header
  # argument

  for (item in boxes) {

    #print(item)

    item %>%
      html_elements('.table-header') %>%
      html_text() %>%
      trimStringArray() -> name

    if (length(name) != 0 && grepl(tolower(game_name), tolower(name))) {
      box = item
      break
    }
  }

  box

  if(!is.null(box)) {
      # Obtain the responsive_table

      box %>%
          html_elements('.responsive-table') -> responsive_table

      # print the responsive-table

      responsive_table

      # scrape the url with the slugs and club ids of the countries

      responsive_table %>%
          html_elements('tr td:first-of-type a') %>%
          html_attr('href') %>%
          strsplit('/') -> raw_url_data

      print(raw_url_data)
      
      raw_url_data

      # obtain the slug of all countries

      raw_url_data[1][[1]][2]
      raw_url_data[1][[1]][length(raw_url_data[1][[1]])]

      for (i in 1:length(raw_url_data)) {
          slugs = c(slugs, raw_url_data[i][[1]][2])
          club_ids = c(club_ids, raw_url_data[i][[1]][length(raw_url_data[1][[1]])])
      }

      # print slugs

      slugs

      # print club_ids

      club_ids

      # obtain the columnNames

      responsive_table %>%
          html_elements('th') %>%
          html_text -> columnNames

      # print columnNames

      columnNames

      # add slug and club_id to columnNames

      columnNames = c(columnNames, c('slug', 'club_id'))

      # print columnNames again

      columnNames

      # obtain length of a all true columns

      responsive_table %>%
          html_elements('tbody tr:nth-of-type(1) td') %>%
          length() -> trueColumnsLength

      # print the length of all true columns

      trueColumnsLength

      for (i in 1:trueColumnsLength) {
          # create css selector string
          # td:not([class*="no-border-rechts"]
          string = paste0('tbody tr td:not([class*="no-border-rechts"]):nth-of-type(', i,')')

          # print the css selector string

          string

          # obtain all row elements of specified column

          responsive_table %>%
              html_elements(string) -> html_node

          # print all row elements of specified column

          html_node

          # only add the values if they are valid!!!

          if(length(html_node) != 0) {
              # obtain the children (if any) from the row elements
              # of specified column

              html_node %>%
                  html_children() -> children

              # print the children

              children

              # check the number of children contained within

              num_of_children = length(children)

              # print the number of children (if any)

              num_of_children

              # check if a child contains any subchildren

              children %>%
                  html_children() -> subchildren

              # print subchildren

              subchildren

              # check the number of children contained within

              num_of_subchildren = length(subchildren)

              # print number of subchildren

              num_of_subchildren

              # dynammically apply a search for the values
              # according to the number of children

              if (num_of_subchildren > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a > span') %>%
                      html_text() -> values
              } else if (num_of_children > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a') %>%
                      html_attr('title') -> values
              } else {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_text -> values
              }

              # wrap the obtained values within a list

              list(values)

              # iterate over all children by

              values = trimStringArray(values)

              # Add this column to the columnValues list

              columnValues = append(columnValues, list(values))
          }
          else {
              print(paste0("Column ", i, " is not a valid value"))
          }

      }

      # add club_id and slug to values

      columnValues = append(columnValues, list(slugs))
      columnValues = append(columnValues, list(club_ids))

      # print columnValues

      columnValues

      # return list
      list(columnNames, columnValues)
  } else {
      # return NULL
      NULL
  }
}

# create Dataframe from scrapeColumnsWM

In [227]:
create_dataframeWM = function(name) {
  df_wm = scrapeColumnsWM(name)

  if(!(is.null(df_wm))) {
      columns = df_wm[[2]]
      columnNames = df_wm[[1]]

      #print(columnNames)

      df_wm = data.frame(columns[[1]])

      for (i in 2:length(columns)) {
          df_wm[i] = columns[[i]]
      }

      names(df_wm) = columnNames[2:length(columnNames)]
      df_wm
  } else {
      NULL
  }
}

In [228]:
df_wm = create_dataframeWM('teilnehmende teams an der wm')
df_wm

[[1]]
[1] ""           "england"    "startseite" "verein"     "3299"      

[[2]]
[1] ""           "frankreich" "startseite" "verein"     "3377"      

[[3]]
[1] ""           "brasilien"  "startseite" "verein"     "3439"      

[[4]]
[1] ""            "deutschland" "startseite"  "verein"      "3262"       

[[5]]
[1] ""           "portugal"   "startseite" "verein"     "3300"      

[[6]]
[1] ""           "spanien"    "startseite" "verein"     "3375"      

[[7]]
[1] ""            "argentinien" "startseite"  "verein"      "3437"       

[[8]]
[1] ""            "niederlande" "startseite"  "verein"      "3379"       

[[9]]
[1] ""           "uruguay"    "startseite" "verein"     "3449"      

[[10]]
[1] ""           "serbien"    "startseite" "verein"     "3438"      

[[11]]
[1] ""           "kroatien"   "startseite" "verein"     "3556"      

[[12]]
[1] ""           "belgien"    "startseite" "verein"     "3382"      

[[13]]
[1] ""           "senegal"    "startseite" "verein"     "3499" 

Club,Squad,&oslash-Age,WC particip.,Foreigners,Market Value,&oslash-Market Value,slug,club_id
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
England,27,25.6,16,11.1 %,€1.29bn,€47.89m,england,3299
France,25,26.9,16,76.0 %,€1.06bn,€42.40m,frankreich,3377
Brazil,26,27.2,22,88.5 %,€963.00m,€37.04m,brasilien,3439
Germany,25,26.6,20,28.0 %,€824.50m,€32.98m,deutschland,3262
Portugal,24,27.3,8,87.5 %,€772.00m,€32.17m,portugal,3300
Spain,24,25.9,16,29.2 %,€751.00m,€31.29m,spanien,3375
Argentina,32,27.7,18,93.8 %,€695.00m,€21.72m,argentinien,3437
Netherlands,25,26.7,11,48.0 %,€494.40m,€19.78m,niederlande,3379
Uruguay,41,26.9,14,58.5 %,€478.40m,€11.67m,uruguay,3449


In [230]:
teilnehmer = df_wm$Club
teilnehmer

[1] "England"       "France"        "Brazil"        "Germany"      
 [5] "Portugal"      "Spain"         "Argentina"     "Netherlands"  
 [9] "Uruguay"       "Serbia"        "Croatia"       "Belgium"      
[13] "Senegal"       "Denmark"       "Poland"        "Switzerland"  
[17] "Morocco"       "United States" "Ghana"         "Canada"       
[21] "Cameroon"      "Wales"         "Mexico"        "Ecuador"      
[25] "Australia"     "Tunisia"       "Saudi Arabia"  "Japan"        
[29] "South Korea"   "Qatar"         "Costa Rica"    "Iran"

# Scrape Columns
We wrote a function, that takes a url to scrape all tables

In [233]:
scrapeGame = function(year, country, club_id, game_name) {

  # base url

  url = paste0("https://www.transfermarkt.com/",
               country,
               "/spielplan/verein/",
               club_id,
               "/plus/0?saison_id=", year)
    
  # print url
  print(url)
    
  # get status code
  res = httr::GET(url)

  # read response
  code = httr::status_code(res)
    
  # print status code
  print(code)
    
  # read url

  url %>%
    read_html() -> html

  # variables

  columnValues = list() # a list containing all values
  columnNames = NULL
  box = NULL # searching for the table_header

  # Obtain the correct responsive-table by name of table_header

  html %>%
    html_elements('.box') -> boxes

  # Print the table headers

  boxes

  # Iterate over each table header to find the match with table_header
  # argument

  for (item in boxes) {

    #print(item)

    if(length(html_elements(item, '.table-header') %>% html_children())!= 0) {
      item %>%
        html_elements('.table-header a img') %>%
        html_attr('title') -> name

      #print(name)
    } else {
      item %>%
        html_elements('.table-header') %>%
        html_text() %>%
        trimStringArray() -> name
        
        #print(name)
    }

    if (length(name) != 0 && grepl(tolower(game_name), tolower(name))) {
      box = item
      break
    }
  }

  box

  if(!is.null(box)) {
      # Obtain the responsive_table

      box %>%
          html_elements('.responsive-table') -> responsive_table

      # print the responsive-table

      responsive_table

      # obtain the columnNames

      responsive_table %>%
          html_elements('th') %>%
          html_text -> columnNames

      # print columnNames

      columnNames
      
      # add game_name to columnNames
      
      columnNames = append(columnNames, 'game_name')
      columnNames = append(columnNames, 'country')

      # obtain length of a all true columns

      responsive_table %>%
          html_elements('tbody tr:nth-of-type(1) td') %>%
          length() -> trueColumnsLength

      # print the length of all true columns

      trueColumnsLength

      for (i in 1:trueColumnsLength) {
          # create css selector string
          # td:not([class*="no-border-rechts"]
          string = paste0('tbody tr td:not([class*="no-border-rechts"]):nth-of-type(', i,')')

          # print the css selector string

          string

          # obtain all row elements of specified column

          responsive_table %>%
              html_elements(string) -> html_node

          # print all row elements of specified column

          html_node

          # only add the values if they are valid!!!

          if(length(html_node) != 0) {
              # obtain the children (if any) from the row elements
              # of specified column

              html_node %>%
                  html_children() -> children

              # print the children

              children

              # check the number of children contained within

              num_of_children = length(children)

              # print the number of children (if any)

              num_of_children

              # check if a child contains any subchildren

              children %>%
                  html_children() -> subchildren

              # print subchildren

              subchildren

              # check the number of children contained within

              num_of_subchildren = length(subchildren)

              # print number of subchildren

              num_of_subchildren

              # dynammically apply a search for the values
              # according to the number of children

              if (num_of_subchildren > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a > span') %>%
                      html_text() -> values
              } else if (num_of_children > 0) {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_elements('a') %>%
                      html_attr('title') -> values
              } else {
                  responsive_table %>%
                      html_elements(string) %>%
                      html_text -> values
              }

              # wrap the obtained values within a list

              list(values)

              # iterate over all children by

              values = trimStringArray(values)

              # Add this column to the columnValues list

              columnValues = append(columnValues, list(values))
          }
          else {
              #print(paste0("Column ", i, " is not a valid value"))
          }

      }
        
      # add game_name to columnValues
      
      columnValues = append(columnValues, name)
      
      # add country name to columnValues
      columnValues = append(columnValues, country)
      
      # return data frame
      # change column names by replacing whitespace with underscore
      columnNames = gsub(" ", "_", columnNames)

      # create a dataframe with first colum and append the rest (workaround)
      df = data.frame(columnValues[[1]])

      for (i in 2:length(columnValues)) {
          df[i] = columnValues[[i]]
      }

      names(df) = columnNames
      return(df)
      
  } else {
      return(data.frame())
  }
}

# Testing Section

In [239]:
#scrapeGame(year = '2009', country = 'England', club_id = 3299, game_name = "weltmeisterschaft [0-9]+")

scrapeGameByYear = function(year, country) {
    
    df_wm %>%
        filter(Club == country) -> tempDf
    
    tempDf$club_id -> club_id
    
    #pmap_df(list('2015', 'England', '3299', c('weltmeisterschaft [0-9]+', 'freundschaftsspiele', 'em-qualifikation', 'wm-qualifikation europa', 'europameisterschaft [0-9]+')), scrapeGame)
    return(pmap_df(list(year, country, club_id,
                        c(
                            'world cup [0-9]+',
                            'international friendlies'
                            #'em-qualifikation'
                            #'wm-qualifikation europa'
                            #'europameisterschaft [0-9]+',
                            #'ostasienmeisterschaft',
                            #'wm-qualifikation asien',
                            #'asienmeisterschaft [0-9]+',
                            #'asienmeisterschaft qualifikation'
                            #'copa [a-z]+ [0-9]+',
                            #'fifa arab cup',
                            #'gold cup [0-9]+'
                        )), scrapeGame))
}

In [241]:
scrapeSomething = function(country) {
    return(map2_df(seq('2009', '2021', by=1), country, scrapeGameByYear))
}

## Scrape world cup & friendlies for all 32 countries

In [242]:
complete_df = map_df(teilnehmer, scrapeSomething)

[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2009"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2009"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2010"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2010"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2011"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2011"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2012"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2012"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2013"
[1] 200
[1] "https://www.transfermarkt.com/England/spielplan/verein/3299/plus/0?saison_id=2013"
[1] 200
[1] "https://www.transfermarkt.com/Engla

ERROR: Error in open.connection(x, "rb"): HTTP error 500.


In [237]:
unique(complete_df$country)

Warning message:
“Unknown or uninitialised column: `country`.”


NULL

# Split Result in Goals

In [16]:
#df_football %>%
#    mutate(goals_scored_england = ternary_v(strsplit_custom_v(Ergebnis, ':', 1), strsplit_custom_v(Ergebnis, ':', 2), (Heimmannschaft=='England'))) -> df_football_goals
#df_football_goals

## We save all data for the WorldCups&friendlies as csv

In [91]:
#complete_df
write.csv2(complete_df, file='./football_em_qualification.csv', row.names = F)

## Modelling - Data Cleaning and Transformation

In [117]:
data_set = read.csv('https://raw.githubusercontent.com/denisnaumov0506/FootballAnalyticsProject/master/world_cup%26friendlies_football.csv', sep=';')
fifa_ranking = read.csv("https://raw.githubusercontent.com/denisnaumov0506/FootballAnalyticsProject/master/FIFA_Ranking.csv", header = TRUE, sep = ",", stringsAsFactors=FALSE, fileEncoding="latin1")


In [213]:
fifa_ranking %>%
    separate(Rang, c('Rang', 'Sym')) %>%
    select(-'Sym') %>%
    mutate(fifa_date = ymd(Datum)) %>%
    select(-c('Datum')) -> fifa_ranking1
fifa_ranking1

date_list = unique(fifa_ranking1$fifa_date)
date_list

Land,Rang,Punkte,fifa_date
<chr>,<chr>,<chr>,<date>
Spain,1,1588,2009-09-02
Netherlands,2,1376,2009-09-02
Germany,3,1186,2009-09-02
Italy,4,1186,2009-09-02
Russia,5,1129,2009-09-02
England,6,1127,2009-09-02
Argentina,7,1113,2009-09-02
Croatia,8,1101,2009-09-02
France,9,1040,2009-09-02


[1] "2009-09-02" "2009-10-16" "2009-11-20" "2009-12-16" "2010-02-03"
  [6] "2010-03-03" "2010-03-31" "2010-04-28" "2010-05-26" "2010-07-14"
 [11] "2010-08-11" "2010-09-24" "2010-10-20" "2010-11-17" "2010-12-15"
 [16] "2011-01-12" "2011-02-02" "2011-03-09" "2011-04-13" "2011-05-18"
 [21] "2011-06-29" "2011-07-27" "2011-08-24" "2011-09-21" "2011-10-19"
 [26] "2011-11-23" "2011-12-21" "2012-01-18" "2012-02-15" "2012-03-07"
 [31] "2012-04-11" "2012-05-09" "2012-06-06" "2012-07-04" "2012-08-08"
 [36] "2012-09-05" "2012-10-03" "2012-11-07" "2012-12-19" "2013-01-17"
 [41] "2013-02-14" "2013-03-14" "2013-04-11" "2013-05-09" "2013-06-06"
 [46] "2013-07-04" "2013-08-08" "2013-09-12" "2013-10-17" "2013-11-28"
 [51] "2013-12-19" "2014-01-16" "2014-02-13" "2014-03-13" "2014-04-10"
 [56] "2014-05-08" "2014-07-17" "2014-08-14" "2014-09-18" "2014-10-23"
 [61] "2014-11-27" "2014-12-18" "2015-02-12" "2015-03-12" "2015-04-09"
 [66] "2015-05-07" "2015-06-04" "2015-07-09" "2015-08-06" "2015-09-03"
 [71] "2015-10-01" "2015-11-05" "2015-12-03" "2016-01-07" "2016-02-04"
 [76] "2016-03-03" "2016-04-07" "2016-05-05" "2016-05-20" "2016-06-02"
 [81] "2016-07-14" "2016-08-11" "2016-09-15" "2016-10-20" "2016-11-24"
 [86] "2016-12-22" "2017-01-12" "2017-02-09" "2017-03-09" "2017-04-06"
 [91] "2017-05-04" "2017-06-01" "2017-07-06" "2017-08-10" "2017-09-14"
 [96] "2017-10-16" "2017-11-23" "2017-12-21" "2018-01-18" "2018-02-15"
[101] "2018-03-15" "2018-04-12" "2018-05-17" "2018-06-07" "2018-08-16"
[106] "2018-09-20" "2018-10-25" "2018-11-29" "2018-12-20" "2019-02-08"
[111] "2019-04-04" "2019-07-25" "2019-09-10" "2019-09-11" "2019-10-24"
[116] "2019-11-28" "2020-02-20" "2020-03-24" "2020-07-23" "2020-09-17"
[121] "2020-10-22" "2020-11-27" "2020-12-10" "2021-02-03" "2021-02-18"
[126] "2021-06-15" "2021-08-12" "2021-10-20" "2021-10-21" "2021-11-19"
[131] "2021-12-23" "2022-02-10" "2022-03-31" "2022-06-06" "2022-06-23"

In [200]:
data_set_reduced = data_set %>%
    select(c('Datum', 'Ort', 'Gegner', 'Ergebnis', 'game_name', 'country'))

## Obtain goals of enemy and country

In [201]:
data_set_reduced %>%
    separate(Ergebnis, c('ergebnis_left', 'ergebnis_right')) %>%
    mutate(goals_country = ternary_v(ergebnis_left, ergebnis_right, Ort=='H')) %>%
    mutate(goals_enemy = ternary_v(ergebnis_left, ergebnis_right, Ort=='A')) -> data_set_increased

Warning message:
“Expected 2 pieces. Additional pieces discarded in 38 rows [51, 53, 212, 302, 305, 417, 471, 528, 530, 531, 583, 611, 612, 654, 687, 823, 824, 830, 831, 832, ...].”


In [202]:
data_set_increased %>%
    separate(Datum, c('weekday', 'Datum_new'), sep=' ') -> test

test %>%
    mutate(date_of_play = dmy(Datum_new)) %>%
    select(-c('weekday', 'Datum_new')) -> data_set_cleaned1

In [203]:
test_date = function(date_param) {
    for(i in 1:length(date_list)) {
        if(date_list[i] > date_param) {
            break;
        }
    }
    if(i != 1) {
        return(date_list[i-1])
    } else {
        return(date_list[1])
    }
}

test_date_v = Vectorize(test_date)

In [204]:
test = test_date(ymd('2013-06-02'))
test

[1] "2013-05-09"

In [205]:
data_set_cleaned1 %>%
    mutate(fifa_date = as_date(test_date_v(date_of_play))) -> data_set_fifa_date

In [206]:
data_set_fifa_date

Ort,Gegner,ergebnis_left,ergebnis_right,game_name,country,goals_country,goals_enemy,date_of_play,fifa_date
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<date>
H,Vereinigte Staaten,1,1,Weltmeisterschaft 2010,england,1,1,2010-06-12,2010-05-26
H,Algerien,0,0,Weltmeisterschaft 2010,england,0,0,2010-06-18,2010-05-26
A,Slowenien,0,1,Weltmeisterschaft 2010,england,1,0,2010-06-23,2010-05-26
A,Deutschland,4,1,Weltmeisterschaft 2010,england,1,4,2010-06-27,2010-05-26
H,Ägypten,3,1,Freundschaftsspiele,england,3,1,2010-03-03,2010-03-03
H,Mexiko,3,1,Freundschaftsspiele,england,3,1,2010-05-24,2010-04-28
H,Japan,2,1,Freundschaftsspiele,england,2,1,2010-05-30,2010-05-26
H,Ungarn,2,1,Freundschaftsspiele,england,2,1,2010-08-11,2010-08-11
H,Frankreich,1,2,Freundschaftsspiele,england,1,2,2010-11-17,2010-11-17


## join both data frames using fifa_date

In [210]:
merge(data_set_fifa_date, fifa_ranking1, by.x=c('country', 'fifa_date'), by.y=c('Land', 'fifa_date')) -> newest_df

In [211]:
newest_df %>%
    select(-c('Punkte'))

country,fifa_date,Ort,Gegner,ergebnis_left,ergebnis_right,game_name,goals_country,goals_enemy,date_of_play,Rang
<chr>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<chr>


## Write function that maps a fifa ranking to each date of data_set_cleaned1

In [ ]:
data_set_cleaned1

## Combine fifa ranking and data_set_increased


In [122]:
bind_cols(data_set_increased, fifa_ranking1)

ERROR: [1m[33mError[39m in [1m[1m`bind_cols()`:[22m
[33m![39m Can't recycle `..1` (size 2359) to match `..2` (size 27931).
